In [2]:
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.colors import Normalize
import matplotlib.ticker as ticker
from datetime import datetime 
from matplotlib.colors import LinearSegmentedColormap



In [3]:
# Shape file code
city_path = r'file_path'
ind_map = gpd.read_file(os.path.join(city_path, 'shape_file_path'))
ind_map.drop([0, 4,  6, 7, 16, 24], inplace = True) #Drop the small UTs
ind_map.set_index(['st_nm'],inplace=True)



In [ ]:
#Code

peak_time = pd.read_csv(r"data_1.csv", index_col=0)
demand_data = pd.read_csv("data_2.csv",index_col=0)
pd.to_datetime(demand_data.index)
demand_data.index = pd.to_datetime(demand_data.index)


demand_data = demand_data[demand_data.index.year == 2022]
demand_data.index = pd.date_range(start = '2022-01-01 00:00:00',end='2022-12-31 23:00:00',freq='H')


# Create a new DataFrame
peak_demand_df = pd.DataFrame(index=pd.date_range(start="2022-01-01",end="2022-12-31"), columns=demand_data.columns)


for state in peak_demand_df.columns:
    for i in peak_demand_df.index:
        peak_demand_df.loc[i,state]= demand_data[(demand_data.index.day==i.day)&(demand_data.index.month==i.month)][state].idxmax().hour
     


day_values = pd.DataFrame(index = peak_demand_df.columns, columns=['day_perc'])

for state in peak_demand_df.columns:
    day_values.loc[state]=peak_demand_df[state].value_counts().sort_index(ascending=True)[5:18].sum()/365
    
day_values['night_perc'] = 1 - day_values['day_perc']     



# Load the shapefile data for India
ind_map = gpd.read_file(os.path.join(city_path, 'Indian_States.shp'))
ind_map.drop([0, 4, 6, 7, 16, 24], inplace=True)  # Drop the small UTs
ind_map.set_index(['st_nm'], inplace=True)

ne_states = ['Meghalaya', 'Manipur', 'Nagaland', 'Assam', 'Arunanchal Pradesh', 'Mizoram', 'Tripura']
piecolors = ['#ffb034', '#000000']





peak_hours = peak_time['2022']


# Define custom color values for different time ranges
day_color = '#FFFB72'  # Yellow (06-18)
evening_color = '#D3D3D3'  # Light grey (18-22)
night_color = '#85C1E9'  # Blue (22-06)

# Create a custom colormap with specific color values for each time range
colors = [night_color] * 6 + [day_color] * 12 + [evening_color] * 5 +  [night_color] * 1
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)



# Normalize the peak hours to the range of 0 to 1
norm = Normalize(vmin=peak_hours.min(), vmax=peak_hours.max())

# Map the peak hours to the colormap values
color_values = norm(peak_hours)




# Create a subset of the shapefile for northeastern states
ne_shape = ind_map.loc[ind_map.index.isin(ne_states)]

# Dissolve the subset into a single polygon
ne_boundary = ne_shape.unary_union


# Plot the map of India with colored states
fig, ax = plt.subplots(figsize=(10, 10))


# Create a subset of the shapefile for northeastern states
ne_shape = ind_map.loc[ind_map.index.isin(ne_states)]

# Dissolve the subset into a single polygon
ne_boundary = ne_shape.unary_union

# Plot the map of India excluding the northeastern states with a constant line width
ind_map.plot(column=color_values, cmap=cmap, linewidth=0.8, edgecolor='black', legend=False, ax=ax)


# Plot the dissolved boundary of the northeastern region with a constant line width
gpd.GeoSeries(ne_boundary).boundary.plot(ax=ax, color='black', linewidth=0.8)

x_coord = ne_boundary.centroid.x
y_coord = ne_boundary.centroid.y
wedges, _ = ax.pie([0.7424657534246575, 0.2575342465753425], colors=piecolors, startangle=90, radius=0.8, center=(x_coord, y_coord))



# Set the aspect ratio to equal for a proper map view
ax.set_aspect('equal')

# Remove axis ticks and labels
ax.set_xticks([])
ax.set_yticks([])

# Add a title
plt.title('2022', fontsize=20)
color_scale_ticks = list(range(0, 24))

# Add colorbar with correct representation of hours
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
cbar = plt.colorbar(sm, ticks=color_scale_ticks, shrink=0.8)
cbar.set_label('Hour of the Day')


# Set the tick intervals
cbar.set_ticks(color_scale_ticks[::1])  # Adjust the slice [::2] for desired tick intervals

# Format the tick labels
cbar.ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))

cbar.set_label('Hour of the Day')


#Pie Plot below

ne_states = ['Meghalaya', 'Manipur', 'Nagaland', 'Assam', 'Arunanchal Pradesh', 'Mizoram', 'Tripura']



piecolors = ['#ffb034', '#000000']


# Iterate over each state
for state in ind_map.index:
    if state not in ne_states:
        # Get the percentages for the current state
        
        percentages = [day_values.loc[state,'day_perc'],day_values.loc[state,'night_perc'] ]

        # Calculate the centroid of the current state
        centroid = ind_map[ind_map.index == state].centroid.squeeze()

        # Plot the pie chart at the centroid coordinates
        wedges, _ = ax.pie(percentages, colors=piecolors, startangle=90, radius=0.8, center=(centroid.x, centroid.y))

        

# Set the aspect ratio to equal for a proper map view
ax.set_aspect('equal')

# Adjust the limits of the plot to include the entire map
ax.set_xlim(ind_map.total_bounds[0], ind_map.total_bounds[2])
ax.set_ylim(ind_map.total_bounds[1], ind_map.total_bounds[3])

plt.show()
